# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bawku,11.0616,-0.2417,33.39,38,99,4.45,GH,1712801542
1,1,kristiansund,63.1115,7.7320,3.36,87,40,3.09,NO,1712801542
2,2,mar del plata,-38.0023,-57.5575,14.49,70,75,8.05,AR,1712801524
3,3,port-aux-francais,-49.3500,70.2167,6.93,95,100,6.40,TF,1712801543
4,4,princeville,45.7668,-61.2985,-0.94,81,0,0.61,CA,1712801543


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
import holoviews as hv
from holoviews.element.tiles import OSM

# Configure the map plot
map_humidity = city_data_df.hvplot.points(
    "Lng",             # Adjust column name for longitude if necessary
    "Lat",             # Adjust column name for latitude if necessary
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",   # Size of the point represents humidity
    color="City",      # Use 'City' column for colors
    cmap='Category10', # Choose a colormap with distinct colors for categories
    alpha=0.7          # Set transparency to 0.7
).opts(
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map plot
map_humidity



c:\Users\Sarah Son Kim\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Filter cities based on criteria (e.g., temperature between 20°C and 25°C, humidity below 50%, cloudiness below 30%, wind speed below 5 m/s)
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] >= 10) & (city_data_df['Max Temp'] <= 25) & 
                                  (city_data_df['Humidity'] < 50) & (city_data_df['Cloudiness'] < 50) & 
                                  (city_data_df['Wind Speed'] < 5)].copy()

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data
filtered_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,crane,31.3974,-102.3501,18.30,25,0,4.33,US,1712801314
62,62,portland,45.5234,-122.6762,18.32,45,0,3.09,US,1712800937
63,63,el matama,16.7095,33.3565,24.12,15,0,4.34,SD,1712801549
113,113,zahedan,29.4963,60.8629,12.95,41,0,3.09,IR,1712801554
122,122,forbes,-33.3833,148.0167,19.63,49,0,1.13,AU,1712801555


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
26,crane,US,31.3974,-102.3501,25,
62,portland,US,45.5234,-122.6762,45,
63,el matama,SD,16.7095,33.3565,15,
113,zahedan,IR,29.4963,60.8629,41,
122,forbes,AU,-33.3833,148.0167,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:

# Set parameters to search for a hotel
radius = 10000  # Define the search radius in meters
params = {
    "categories": "accommodation.hotel",
    "radius": radius,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
hotel_df.head()

Starting hotel search
crane - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
el matama - nearest hotel: الكوثر
zahedan - nearest hotel: هتل صالح;بانک ملی
forbes - nearest hotel: No hotel found
jiuquan - nearest hotel: 民族宾馆
yinchuan - nearest hotel: Yinchuan hotel
shaydon - nearest hotel: No hotel found
lovington - nearest hotel: Comfort Inn & Suites
baiyin - nearest hotel: 红鹭宾馆
jinchang - nearest hotel: No hotel found
bam - nearest hotel: هتل جهانگردی بم
chyorny yar - nearest hotel: Гостиница
cootamundra - nearest hotel: Globe Hotel
cabo san lucas - nearest hotel: Comfort Rooms
salida - nearest hotel: Hampton Inn & Suites
safi - nearest hotel: No hotel found
beni mellal - nearest hotel: Tasmmet
el hadjira - nearest hotel: No hotel found
midelt - nearest hotel: Avenue Molay Idriss
tamanrasset - nearest hotel: فندق الأمان
chinde - nearest hotel: Kanyum cottage village
kharkhorin - nearest hotel: 점심식사 good restaurant.
eagle - nearest hotel: Hilton Garden Inn
frankenmut

,City,Country,Lat,Lng,Humidity,Hotel Name
26,crane,US,31.3974,-102.3501,25,No hotel found
62,portland,US,45.5234,-122.6762,45,the Hoxton
63,el matama,SD,16.7095,33.3565,15,الكوثر
113,zahedan,IR,29.4963,60.8629,41,هتل صالح;بانک ملی
122,forbes,AU,-33.3833,148.0167,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot with hotel names and country information in the hover message
map_with_hotel_info = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],  # Add hotel name and country to hover message
    color="City",
    cmap='Category10', # Choose a colormap with distinct colors for categories
    alpha=0.7
).opts(
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map plot
map_with_hotel_info


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)